In [1]:
import cobra
import GEMS
import pandas as pd
import numpy as np
import os
import re


import pickle
from etcpy import etc
from etcpy import tempDep
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import r2_score

import matplotlib.pyplot as plt

In [2]:
def aerobic(params):
    # thermalParams: a dictionary with ids like uniprotid_Topt 
    df = etc.calculate_thermal_params(params)
    #mae = pickle.load(open(os.path.join(path,'models/aerobic.pkl'),'rb'))
    mae = cobra.io.load_matlab_model('../models/ecModel.mat')
    
    match = re.compile('usage_prot_*', re.IGNORECASE)
    for react in mae.reactions:
        if match.match(react.id) and react.lower_bound != -1000:
            ori_lb = react.lower_bound
            react.lower_bound = ori_lb*2.5


    mae.reactions.EX_glc__D_e.lower_bound = -100
    mae.reactions.BIOMASS_Ec_iJO1366_core_53p95M.upper_bound = 50
    #mae.medium['EX_glc__D_e'] = 100
    
    try: 
        rae, gae, oae, cae, aae = etc.simulate_growth(mae,dfae_batch.index+273.15,df=df,sigma=0.5)
    except: 
        rae = np.zeros(dfae_batch.shape[0])
        model = None
    
    rae = [0 if x is None else x for x in rae]
    rae = [0 if x<1e-3 else x for x in rae]
    rexp = dfae_batch['r'].values
    #print(rae)
    print(abs(rae - dfae_batch['r'].values))
    print('r2_batch:',r2_score(rexp,rae))
    print('MSE_ae',MSE(rexp,rae))
    return {'growth':np.array(rae), 'glucose':np.array(gae), 'oxygen': np.array(oae), 'carbonDioxide': np.array(cae), 'acetate': np.array(aae)}, r2_score(rexp,rae), MSE(rexp,rae)

In [3]:
data = GEMS.load_exp_data('../data/ExpGrowth.csv')
path = '../'
params = pd.read_csv(os.path.join(path,'data/BestParamsTopt129.csv'),index_col=0)
params

,Topt,Topt_std,Length,Tm,Tm_std,T90,dCpt,dCpt_std,topt_source,TmTag
P77467,312.581885,10,262,315.525470,7.588735,NaN,-4934.391324,1000,predicted,Mean
P76077,316.249662,10,309,341.198282,7.588735,NaN,-3031.895628,1000,predicted,Mean
P76078,316.889489,10,95,336.876992,7.588735,NaN,-4970.916037,1000,BullShit,Mean
P76079,315.569489,10,248,351.858297,7.588735,NaN,-2296.738492,1000,predicted,Mean
P76081,317.436480,10,356,330.800735,7.588735,NaN,-3212.147663,1000,BullShit,Mean
...,...,...,...,...,...,...,...,...,...,...
P0AG80,316.761813,10,438,324.738737,4.070000,330.45,-5800.075533,1000,predicted,Exp
P77718,309.573269,10,482,319.125751,4.070000,320.15,-5078.319618,1000,predicted,Exp
P0A6D5,314.182243,10,288,317.369952,7.710000,NaN,-4036.145478,1000,predicted,Mean
standard,313.546929,10,272,325.206318,7.710000,NaN,-4108.399459,1000,BullShit,Mean


In [4]:
df = pd.DataFrame(columns=['iter', 'temp', 'growth', 'o2flux', 'gluFlux', 'co2flux', 'acetateFlux', 'R2', 'MSE'])
for key in data.keys():
    
    # Extracting the correct data subset
    dfae_batch = data[key].set_index('Ts').rename_axis(None)
    
    # Running the temp dependent model
    rae, score, mse = aerobic(params)
    
    df1 = pd.DataFrame(columns=['iter', 'temp', 'growth', 'o2flux', 'gluFlux', 'co2flux', 'acetateFlux', 'R2', 'MSE'])
    df1['iter'] = [key for j in range(len(dfae_batch.index))]
    df1['temp'] = dfae_batch.index
    df1['growth'] = rae['growth']
    df1['o2flux'] = rae['oxygen']
    df1['gluFlux'] = rae['glucose']
    df1['co2flux'] = rae['carbonDioxide']
    df1['acetateFlux'] = rae['acetate']
    df1['R2'] = [score for j in range(len(dfae_batch.index))]
    df1['MSE'] = [mse for j in range(len(dfae_batch.index))]
    df = pd.concat([df, df1])

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-06
NGAM is: 3.23
Growth at  20.153299075508016 is:  0.18032328463552771
NGAM is: 3.23
Growth at  23.00151111308503 is:  0.3129177514111487
NGAM is: 3.666948749964016
Growth at  26.331896319367047 is:  0.5426830163341987
NGAM is: 4.228097946953468
Growth at  28.251819864188008 is:  0.7118252843695363
NGAM is: 4.675583853845237
Growth at  29.99421176442104 is:  0.8687829042328808
NGAM is: 5.047267123847456
Growth at  31.62212188446705 is:  1.0252208357215702
NGAM is: 5.44939156263788
Growth at  33.61728633658504 is:  1.2230020240506274
NGAM is: 5.950125048399887
Growth at  36.55299482796005 is:  1.4484961183132101
NGAM is: 6.237403245621997
Growth at  38.54155004207803 is:  1.5069344828644733
NGAM is: 6.449817187489966
Growth at  40.20094365636703 is:  1.5219223684103205
NGAM is: 6.681009354243395
Growth at  42.23876395989504 is:  1.3610103623729124
NGAM is: 6.9000236503978725
Growth at  44.4534910975

/home/adi/anaconda3/lib/python3.11/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


Failed to solve the problem
Growth at  6.990780000000029 is:  0
NGAM is: 3.23
Growth at  7.959210000000041 is:  0.006488154422566912
NGAM is: 3.23
Failed to solve the problem
Growth at  8.527750000000026 is:  0
NGAM is: 3.23
Growth at  9.952300000000037 is:  0.013033910644493897
NGAM is: 3.23
Growth at  9.951150000000041 is:  0.013029256523167861
NGAM is: 3.23
Growth at  9.950860000000034 is:  0.013028083108020175
NGAM is: 3.23
Growth at  11.999600000000044 is:  0.024065379386632772
NGAM is: 3.23
Growth at  12.398600000000044 is:  0.0269797241831061
NGAM is: 3.23
Growth at  12.454800000000034 is:  0.02741394604417942
NGAM is: 3.23
Growth at  14.392200000000003 is:  0.04658736035426464
NGAM is: 3.23
Growth at  15.132200000000012 is:  0.05646750817061427
NGAM is: 3.23
Growth at  14.960900000000038 is:  0.05403529382450657
NGAM is: 3.23
Growth at  14.96040000000005 is:  0.05402832654980729
NGAM is: 3.23
Growth at  14.902700000000038 is:  0.053229398181078816
NGAM is: 3.23
Growth at  14.95

In [5]:
df.to_csv('../results/allDataModelResultsBsetTillNow.csv')